In [2]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime, timedelta

## Load data

In [3]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

## create report_dt column
df['report_dt'] = pd.to_datetime(df.REPORT_DAT)

## Warm-up Demo

In [5]:
%%time
for i in range(df.shape[0]):
    r = df.iloc[i]
    r.X + r.Y

CPU times: user 1.38 s, sys: 10.1 ms, total: 1.39 s
Wall time: 1.46 s


In [6]:
%%time
for i,r in df.iterrows():
    r.X + r.Y

CPU times: user 850 ms, sys: 10.3 ms, total: 860 ms
Wall time: 875 ms


In [7]:
%%time
df.apply(lambda r: r.X + r.Y, axis = 1)

CPU times: user 244 ms, sys: 5.92 ms, total: 250 ms
Wall time: 249 ms


0       -38.064623
1       -38.122138
2       -38.050995
3       -38.076144
4       -38.063962
           ...    
27928   -38.135656
27929   -38.118390
27930   -38.089127
27931   -38.099146
27932   -38.089438
Length: 27933, dtype: float64

In [8]:
%%time
## Super fast, but only works with built-in numpy functions.
df.X + df.Y

CPU times: user 919 µs, sys: 66 µs, total: 985 µs
Wall time: 998 µs


0       -38.064623
1       -38.122138
2       -38.050995
3       -38.076144
4       -38.063962
           ...    
27928   -38.135656
27929   -38.118390
27930   -38.089127
27931   -38.099146
27932   -38.089438
Length: 27933, dtype: float64

# Practice time

In [10]:
## define crimes to look for and crimes to look within
## CCN is Central Complaint Number: https://go.mpdconline.com/GO/GO_401_01.pdf
CCN_examples = ['20165648', '20123250']
C_Tar = crimes_lookfor = df[df.CCN.astype(str).isin(CCN_examples)][['CCN', 'WARD', 'OFFENSE', 'report_dt']]
C_Oth = other_crimes = df[~df.CCN.astype(str).isin(CCN_examples)]

## print crimes_lookfor
C_Tar.head()
# other_crimes.head()

,CCN,WARD,OFFENSE,report_dt
4225,20123250,2,MOTOR VEHICLE THEFT,2020-08-29 05:00:25+00:00
7779,20165648,6,MOTOR VEHICLE THEFT,2020-11-20 02:25:50+00:00


**Task**: we have two crimes we want to look for. We want to look in the remaining crime reports for crime reports that are:

- Located in the same ward as the two focal crimes
- Reported at the same time as the focal crime or up to 1000 minutes later (changed from slides which stated 20 mins since crime ids changed since last time so this long bandwidth helps us find matches!)

Solutions compare two ways to solve:

- Using a for loop
- Using a function

## 1. Loop approach

In [18]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(C_Tar.shape[0]): # same as 
    
    ## extract row
    r = one_row = C_Tar.iloc[i]

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(r.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20123250 1371  -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         6317  -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         6763  -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         9838  -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         16403 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 1371       DAY  OTHERS          THEFT/OTHER   
         6317       DAY  OTHERS         THEFT F/AUTO   
         6763       DAY  OTHERS         THEFT F/AUTO   
         9838   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         16403      DAY  OTHERS          THEFT/OTHER   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 1371        700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   
         6317        1700 - 1799 BLOCK OF P STREET NW  396523.0  137976.0   
         6763   2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0  138388.0   
         9838        1700 - 1779 BLOCK OF M STREET NW  396546.0  137533.0   
         16403    1724 - 1799 BLOCK OF 17TH STREET NW  396662.0  138429.0   

                ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  LONGITUDE  \
20123250 1371   ...        5801.0    Precinct 129  38.899121 -77.021926   
         6317   ...        5303.0     Precinct 15  38.909638 -77.040089   
         6763   ...        4100.0     Precinct 13  38.913346 -77.050526   
         9838   ...       10700.0     Precinct 17  38.905648 -77.039822   
         16403  ...        5302.0     Precinct 15  38.913720 -77.038489   

                            BID              START_DATE  \
20123250 1371          DOWNTOWN  2020/08/29 16:05:40+00   
         6317               NaN  2020/08/28 22:00:23+00   
         6763               NaN  2020/08/26 22:00:29+00   
         9838   GOLDEN TRIANGLE  2020/08/27 19:01:24+00   
         16403              NaN  2020/08/28 20:55:00+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20123250 1371   2020/08/29 16:08:33+00  398247584             NaN   
         6317   2020/08/29 08:00:27+00  398455909             NaN   
         6763   2020/08/27 12:00:51+00  398456841             NaN   
         9838   2020/08/29 19:00:05+00  398514253             NaN   
         16403  2020/08/28 21:05:00+00  398535856             NaN   

                               report_dt  
20123250 1371  2020-08-29 17:15:19+00:00  
         6317  2020-08-29 16:05:18+00:00  
         6763  2020-08-29 16:45:57+00:00  
         9838  2020-08-29 22:04:46+00:00  
         16403 2020-08-29 14:29:59+00:00  

[5 rows x 26 columns]

# 1.5 Iterrow Approach

In [19]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i, r in C_Tar.iterrows(): # same as 

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(r.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20123250 1371  -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         6317  -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         6763  -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         9838  -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         16403 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 1371       DAY  OTHERS          THEFT/OTHER   
         6317       DAY  OTHERS         THEFT F/AUTO   
         6763       DAY  OTHERS         THEFT F/AUTO   
         9838   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         16403      DAY  OTHERS          THEFT/OTHER   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 1371        700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   
         6317        1700 - 1799 BLOCK OF P STREET NW  396523.0  137976.0   
         6763   2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0  138388.0   
         9838        1700 - 1779 BLOCK OF M STREET NW  396546.0  137533.0   
         16403    1724 - 1799 BLOCK OF 17TH STREET NW  396662.0  138429.0   

                ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  LONGITUDE  \
20123250 1371   ...        5801.0    Precinct 129  38.899121 -77.021926   
         6317   ...        5303.0     Precinct 15  38.909638 -77.040089   
         6763   ...        4100.0     Precinct 13  38.913346 -77.050526   
         9838   ...       10700.0     Precinct 17  38.905648 -77.039822   
         16403  ...        5302.0     Precinct 15  38.913720 -77.038489   

                            BID              START_DATE  \
20123250 1371          DOWNTOWN  2020/08/29 16:05:40+00   
         6317               NaN  2020/08/28 22:00:23+00   
         6763               NaN  2020/08/26 22:00:29+00   
         9838   GOLDEN TRIANGLE  2020/08/27 19:01:24+00   
         16403              NaN  2020/08/28 20:55:00+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20123250 1371   2020/08/29 16:08:33+00  398247584             NaN   
         6317   2020/08/29 08:00:27+00  398455909             NaN   
         6763   2020/08/27 12:00:51+00  398456841             NaN   
         9838   2020/08/29 19:00:05+00  398514253             NaN   
         16403  2020/08/28 21:05:00+00  398535856             NaN   

                               report_dt  
20123250 1371  2020-08-29 17:15:19+00:00  
         6317  2020-08-29 16:05:18+00:00  
         6763  2020-08-29 16:45:57+00:00  
         9838  2020-08-29 22:04:46+00:00  
         16403 2020-08-29 14:29:59+00:00  

[5 rows x 26 columns]

## 2. Function approach

Practice rewriting the above loop as a function

### 2.1 define the function

In [38]:
store_matches_2 = {}

def find_related_crimes(r): # imagine the function taking in one row as its sole variable
    
    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    CUTOFF = r.report_dt +  timedelta(minutes=1200)

    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches_2[r.CCN] = same_wards_sametime

### 2.2 apply it to one of the focal crimes

In [39]:
find_related_crimes(C_Tar.iloc[0])

In [40]:
## finally, concatenate results into one df
all_matches = pd.concat(store_matches_2)
all_matches

X          Y       CCN              REPORT_DAT  \
20123250 1371  -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         6317  -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         6763  -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         9838  -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         16403 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   
         17492 -77.027565  38.897353  20123609  2020/08/30 00:05:52+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 1371       DAY  OTHERS          THEFT/OTHER   
         6317       DAY  OTHERS         THEFT F/AUTO   
         6763       DAY  OTHERS         THEFT F/AUTO   
         9838   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         16403      DAY  OTHERS          THEFT/OTHER   
         17492  EVENING  OTHERS  MOTOR VEHICLE THEFT   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 1371        700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   
         6317        1700 - 1799 BLOCK OF P STREET NW  396523.0  137976.0   
         6763   2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0  138388.0   
         9838        1700 - 1779 BLOCK OF M STREET NW  396546.0  137533.0   
         16403    1724 - 1799 BLOCK OF 17TH STREET NW  396662.0  138429.0   
         17492       1100 - 1199 BLOCK OF F STREET NW  397609.0  136611.0   

                ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  LONGITUDE  \
20123250 1371   ...        5801.0    Precinct 129  38.899121 -77.021926   
         6317   ...        5303.0     Precinct 15  38.909638 -77.040089   
         6763   ...        4100.0     Precinct 13  38.913346 -77.050526   
         9838   ...       10700.0     Precinct 17  38.905648 -77.039822   
         16403  ...        5302.0     Precinct 15  38.913720 -77.038489   
         17492  ...        5802.0    Precinct 129  38.897346 -77.027563   

                            BID              START_DATE  \
20123250 1371          DOWNTOWN  2020/08/29 16:05:40+00   
         6317               NaN  2020/08/28 22:00:23+00   
         6763               NaN  2020/08/26 22:00:29+00   
         9838   GOLDEN TRIANGLE  2020/08/27 19:01:24+00   
         16403              NaN  2020/08/28 20:55:00+00   
         17492         DOWNTOWN  2020/08/29 23:08:57+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20123250 1371   2020/08/29 16:08:33+00  398247584             NaN   
         6317   2020/08/29 08:00:27+00  398455909             NaN   
         6763   2020/08/27 12:00:51+00  398456841             NaN   
         9838   2020/08/29 19:00:05+00  398514253             NaN   
         16403  2020/08/28 21:05:00+00  398535856             NaN   
         17492                     NaN  398557899             NaN   

                               report_dt  
20123250 1371  2020-08-29 17:15:19+00:00  
         6317  2020-08-29 16:05:18+00:00  
         6763  2020-08-29 16:45:57+00:00  
         9838  2020-08-29 22:04:46+00:00  
         16403 2020-08-29 14:29:59+00:00  
         17492 2020-08-30 00:05:52+00:00  

[6 rows x 26 columns]

### 2.3 Use apply to cover all the other focal crimes

In [33]:
C_Tar.apply(find_related_crimes, axis = 1)

4225    None
7779    None
dtype: object